# Лабораторная работа № 4
### Провести классификацию найденного датасета, методами наивного Байеса. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.


## - Импорт библиотек

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

## - Инициализация датасета
#### (Датасет представляет собой данные о прошедших матчах в Бразильском CS:GO)

In [2]:
file_path = r'C:\Users\thebi\tb_lobby_stats_player.csv'

data = pd.read_csv(file_path)

num_rows, num_cols = data.shape
print(f"Количество строк: {num_rows}")
print(f"Количество столбцов: {num_cols}")


Количество строк: 184152
Количество столбцов: 38


In [3]:
print(data.columns)

Index(['idLobbyGame', 'idPlayer', 'idRoom', 'qtKill', 'qtAssist', 'qtDeath',
       'qtHs', 'qtBombeDefuse', 'qtBombePlant', 'qtTk', 'qtTkAssist',
       'qt1Kill', 'qt2Kill', 'qt3Kill', 'qt4Kill', 'qt5Kill', 'qtPlusKill',
       'qtFirstKill', 'vlDamage', 'qtHits', 'qtShots', 'qtLastAlive',
       'qtClutchWon', 'qtRoundsPlayed', 'descMapName', 'vlLevel', 'qtSurvived',
       'qtTrade', 'qtFlashAssist', 'qtHitHeadshot', 'qtHitChest',
       'qtHitStomach', 'qtHitLeftAtm', 'qtHitRightArm', 'qtHitLeftLeg',
       'qtHitRightLeg', 'flWinner', 'dtCreatedAt'],
      dtype='object')


## - Удаление столбцов с необрабатываемыми типами данных или просто не влияющих на исход матча

In [4]:
columns_to_drop = ['idRoom', 'idPlayer', 'idLobbyGame', 'qtFlashAssist', 'qtHitHeadshot', 'qtHitChest',
       'qtHitStomach', 'qtHitLeftAtm', 'qtHitRightArm', 'qtHitLeftLeg',
       'qtHitRightLeg', 'dtCreatedAt', 'descMapName']
data = data.drop(columns=columns_to_drop)

In [5]:
print(data.columns)

Index(['qtKill', 'qtAssist', 'qtDeath', 'qtHs', 'qtBombeDefuse',
       'qtBombePlant', 'qtTk', 'qtTkAssist', 'qt1Kill', 'qt2Kill', 'qt3Kill',
       'qt4Kill', 'qt5Kill', 'qtPlusKill', 'qtFirstKill', 'vlDamage', 'qtHits',
       'qtShots', 'qtLastAlive', 'qtClutchWon', 'qtRoundsPlayed', 'vlLevel',
       'qtSurvived', 'qtTrade', 'flWinner'],
      dtype='object')


## - Поиск пустых значений в датасете и удаление соответствующих строк

In [6]:
# Урезка датасета
data = data.sample(n=50000, random_state=42)

empty_columns = data.columns[data.isnull().any()]
empty_columns_count = data[empty_columns].isnull().sum()

# Вывод столбцов с пустыми значениями и их количество
if len(empty_columns) > 0:
    print("Столбцы с пустыми значениями и их количество:")
    for column in empty_columns:
        print(f"Столбец '{column}': {empty_columns_count[column]} пустых значений")
    print()

    # Удаление строк с пустыми значениями
    data_cleaned = data.dropna()
    print(f"Удалено строк с пустыми значениями: {len(data) - len(data_cleaned)}")
else:
    print("В датасете нет пустых значений")

Столбцы с пустыми значениями и их количество:
Столбец 'qtTk': 37 пустых значений
Столбец 'qtTkAssist': 37 пустых значений
Столбец 'qtHits': 37 пустых значений
Столбец 'qtLastAlive': 37 пустых значений
Столбец 'qtSurvived': 189 пустых значений
Столбец 'qtTrade': 189 пустых значений

Удалено строк с пустыми значениями: 189


## - Поиск столбцов с нечисловыми значениями

In [7]:
non_numeric_columns = data_cleaned.select_dtypes(exclude=['number']).columns.tolist()

if len(non_numeric_columns) > 0:
    print("Столбцы с нечисловыми значениями:")
    for column in non_numeric_columns:
        print(column)
else:
    print("Все столбцы содержат числовые значения")

# Подсчет количества строк и столбцов
num_rows, num_cols = data_cleaned.shape
print(f"Количество строк: {num_rows}")
print(f"Количество столбцов: {num_cols}")

Все столбцы содержат числовые значения
Количество строк: 49811
Количество столбцов: 25


## - Поиск лучших гиперпараметров

In [10]:
%%time
X = data_cleaned.drop('flWinner', axis=1)
y = data_cleaned['flWinner']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


under_sampler = RandomUnderSampler(random_state=42)


X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)


# Определение методов наивного байеса
naive_bayes_models = {
    'GaussianNB': GaussianNB(),
    'MultinomialNB': MultinomialNB(),
    'BernoulliNB': BernoulliNB(),
}

# Параметры для GridSearchCV
param_grid = {
    'GaussianNB': {},
    'MultinomialNB': {'alpha': [0.1, 0.5, 1.0]},
    'BernoulliNB': {'alpha': [0.1, 0.5, 1.0], 'binarize': [0.0, 0.1, 0.2]},
}

# Выбор метрики для оценки моделей
scoring_metric = 'accuracy'

# Обучение и оценка моделей с использованием GridSearchCV
best_models = {}
for model_name, model in naive_bayes_models.items():
    grid_search = GridSearchCV(model, param_grid[model_name], scoring=scoring_metric, cv=5)
    grid_search.fit(X_resampled, y_resampled)
    
    best_models[model_name] = grid_search.best_estimator_
# Оценка наилучшей модели на тестовом наборе
best_model_name = max(best_models, key=lambda k: grid_search.cv_results_['mean_test_score'][grid_search.best_index_])
best_model = best_models[best_model_name]


# Вывод результатов
print(f"Лучшая модель: {best_model_name}")
print(f"Лучшие параметры: {grid_search.best_params_}")

Лучшая модель: GaussianNB
Лучшие параметры: {'alpha': 0.1, 'binarize': 0.0}
Wall time: 1.4 s


## - Обучение модели 

In [11]:
%%time

best_model.fit(X_resampled, y_resampled)

y_pred_full = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_full)
classification_rep = classification_report(y_test, y_pred_full)
print(f'Точность модели: {accuracy}')
print(classification_rep)

Точность модели: 0.6977164366373902
              precision    recall  f1-score   support

           0       0.71      0.69      0.70     10083
           1       0.69      0.71      0.70      9842

    accuracy                           0.70     19925
   macro avg       0.70      0.70      0.70     19925
weighted avg       0.70      0.70      0.70     19925

Wall time: 54.1 ms


__Из проведенной лабораторной работы по использованию моделей наивного Байеса для классификации следует следующий вывод:__

    Лучшая модель и параметры: В результате подбора гиперпараметров наилучшей моделью оказалась Gaussian Naive Bayes, с оптимальными значениями гиперпараметров: alpha=0.1 и binarize=0.0. Эти параметры позволили модели достичь наилучшей производительности на данном наборе данных.

   Метрики качества модели:

    Точность (Precision) и Полнота (Recall): Для класса 0 точность составляет 71%, полнота - 69%. Для класса 1 точность равна 69%, полнота - 71%. Обе метрики близки друг к другу для обоих классов, что указывает на сбалансированность модели в выявлении обоих классов.

    F1-score: Средневзвешенное значение F1-меры для обоих классов также составляет около 0.70. Эта метрика учитывает как точность, так и полноту модели, и является показателем их сбалансированности.

Общая точность модели: Общая точность модели (accuracy) на тестовой выборке составляет 70%. Это означает, что модель правильно классифицировала 70% всех примеров на тестовой выборке.

В целом, модель Gaussian Naive Bayes показывает приемлемую производительность, достигая сбалансированных значений метрик для обоих классов.